In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
train_df = pd.read_csv('/train.csv')
test_df = pd.read_csv('/test.csv')

In [ ]:
KGAT_88309d32b0ab7b77b3273e3189011e4b

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train_df.head()

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56


In [ ]:
train_df['road_type'].value_counts()

,count
road_type,
highway,173672
rural,172719
urban,171363


In [ ]:
train_df['time_of_day'].value_counts()

NameError: name 'train_df' is not defined

In [ ]:
train_df['lighting'].value_counts()

,count
lighting,
dim,183826
daylight,178015
night,155913


In [ ]:
train_df['weather'].value_counts()

,count
weather,
foggy,181463
clear,179306
rainy,156985


In [ ]:
len(train_df)

517754

In [ ]:
df_dummies = pd.get_dummies(train_df, columns = ['road_type', 'lighting', 'weather', 'time_of_day'], dtype=int)

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [ ]:
X = df_dummies.drop(columns = ['id', 'accident_risk'])
y = df_dummies['accident_risk']

In [ ]:
X['road_signs_present'] = X['road_signs_present'].astype(int)
X['public_road'] = X['public_road'].astype(int)
X['holiday'] = X['holiday'].astype(int)
X['school_season'] = X['school_season'].astype(int)

In [ ]:
X_train = sm.add_constant(X)  # adds intercept term

# Convert all boolean columns to integers
for col in X_train.select_dtypes(include=['bool']).columns:
    X_train[col] = X_train[col].astype(int)

model = sm.OLS(y, X_train).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:          accident_risk   R-squared:                       0.805
Model:                            OLS   Adj. R-squared:                  0.805
Method:                 Least Squares   F-statistic:                 1.336e+05
Date:                Sun, 07 Dec 2025   Prob (F-statistic):               0.00
Time:                        17:06:34   Log-Likelihood:             6.1696e+05
No. Observations:              517754   AIC:                        -1.234e+06
Df Residuals:                  517737   BIC:                        -1.234e+06
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -0

In [ ]:
X_train

,const,num_lanes,curvature,speed_limit,road_signs_present,public_road,holiday,school_season,num_reported_accidents,road_type_highway,...,road_type_urban,lighting_daylight,lighting_dim,lighting_night,weather_clear,weather_foggy,weather_rainy,time_of_day_afternoon,time_of_day_evening,time_of_day_morning
0,1.0,2,0.06,35,0,1,0,1,1,0,...,1,1,0,0,0,0,1,1,0,0
1,1.0,4,0.99,35,1,0,1,1,0,0,...,1,1,0,0,1,0,0,0,1,0
2,1.0,4,0.63,70,0,1,1,0,2,0,...,0,0,1,0,1,0,0,0,0,1
3,1.0,4,0.07,35,1,1,0,0,1,1,...,0,0,1,0,0,0,1,0,0,1
4,1.0,1,0.58,60,0,0,1,0,1,0,...,0,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517749,1.0,4,0.10,70,1,1,0,0,2,1,...,0,1,0,0,0,1,0,1,0,0
517750,1.0,4,0.47,35,1,1,0,0,1,0,...,0,1,0,0,0,0,1,0,0,1
517751,1.0,4,0.62,25,0,0,0,1,0,0,...,1,1,0,0,0,1,0,1,0,0
517752,1.0,3,0.63,25,1,0,1,1,3,1,...,0,0,0,1,1,0,0,1,0,0


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [ ]:
rf = RandomForestRegressor(
    n_estimators=50,
    random_state=42
)

rf.fit(X_train, y_train)

# Predict
y_pred = rf.predict(X_test)

# Evaluate
print("R²:", r2_score(y_test, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

R²: 0.8710445742775084
RMSE: 0.05967195544403465


In [ ]:
print("RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

RMSE: 0.05967195544403465


In [ ]:
len(X_train)

414203

In [ ]:
print(X_train.shape)

(414203, 20)


In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#X_train = X_train.drop(columns = ['const'])
X_train_scaled = scaler.fit_transform(X_train)
Xtest_scaled = scaler.transform(X_test)

In [ ]:
X_train

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import time
import numpy as np

# Define the neural network for PyTorch
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.output_layer = nn.Linear(hidden_size2, 1)

    def forward(self, x):
        x = self.relu1(self.layer1(x))
        x = self.relu2(self.layer2(x))
        x = self.output_layer(x)
        return x

# Select device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Prepare data
# Binarize y_train and y_test for classification, assuming 0.5 as threshold
y_train_binarized = (y_train.values >= 0.5).astype(float)
y_test_binarized = (y_test.values >= 0.5).astype(float)

X_train_pt = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_pt = torch.tensor(y_train_binarized, dtype=torch.float32).reshape(-1, 1)
Xtest_pt = torch.tensor(Xtest_scaled, dtype=torch.float32)
ytest_pt = torch.tensor(y_test_binarized, dtype=torch.float32).reshape(-1, 1)

train_data = TensorDataset(X_train_pt, y_train_pt)
test_data = TensorDataset(Xtest_pt, ytest_pt)

# Control (Base Model) settings
input_size = X_train_scaled.shape[1] # 11 features
hidden_size1 = 64
hidden_size2 = 32
learning_rate = 0.001
batch_size = 32
epochs = 50

model_torch_base = SimpleNN(input_size, hidden_size1, hidden_size2).to(device)
criterion = nn.BCEWithLogitsLoss() # Combines sigmoid and Binary Cross Entropy for stability
optimizer = optim.Adam(model_torch_base.parameters(), lr=learning_rate)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Training loop with timing
start_time = time.time()
history_base_torch = {'loss': [], 'accuracy': [], 'auc': []}

model_torch_base.train()
for epoch in range(epochs):
    running_loss = 0.0
    all_labels = []
    all_predictions = []

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_torch_base(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

        # Store predictions and labels for metrics calculation
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(torch.sigmoid(outputs).detach().cpu().numpy())

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = accuracy_score(np.array(all_labels).round(), np.array(all_predictions).round())
    epoch_auc = roc_auc_score(np.array(all_labels), np.array(all_predictions))

    history_base_torch['loss'].append(epoch_loss)
    history_base_torch['accuracy'].append(epoch_accuracy)
    history_base_torch['auc'].append(epoch_auc)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}, AUC: {epoch_auc:.4f}")

end_time = time.time()
print(f"Total training time: {end_time - start_time:.2f} seconds")

# Evaluation on the test set
model_torch_base.eval()
all_test_labels = []
all_test_predictions_raw = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_torch_base(inputs)
        all_test_labels.extend(labels.cpu().numpy())
        all_test_predictions_raw.extend(outputs.cpu().numpy())

y_pred_torch_base = torch.sigmoid(torch.tensor(all_test_predictions_raw)).numpy().ravel()
y_pred_labels_torch_base = (y_pred_torch_base >= 0.5).astype(int)

# Use the binarized y_test for computing metrics
y_test_1d_torch_final = np.array(y_test_binarized).ravel()

# Compute metrics
accuracy_torch_base  = accuracy_score(y_test_1d_torch_final, y_pred_labels_torch_base)
precision_torch_base = precision_score(y_test_1d_torch_final, y_pred_labels_torch_base)
recall_torch_base    = recall_score(y_test_1d_torch_final, y_pred_labels_torch_base)
f1_torch_base        = f1_score(y_test_1d_torch_final, y_pred_labels_torch_base)
auc_torch_base       = roc_auc_score(y_test_1d_torch_final, y_pred_torch_base)

Base_Model_Metrics_torch = {
    'Experiment' : "Control (Base Model) - PyTorch",
    'Learning Rate': learning_rate,
    'Activation': 'relu',
    'num_layers': 2,
    'Batch Size': batch_size,
    'Epochs': epochs,
    'Early Stopping': 'NO',
    'Accuracy': accuracy_torch_base,
    'Precision': precision_torch_base,
    'Recall': recall_torch_base,
    'F1 Score' : f1_torch_base,
    'AUC': auc_torch_base
}

print("PyTorch Base Model Metrics:")
print(Base_Model_Metrics_torch)


Using device: cuda
Epoch [10/50], Loss: 0.1431, Accuracy: 0.9357, AUC: 0.9786
Epoch [20/50], Loss: 0.1415, Accuracy: 0.9363, AUC: 0.9791
Epoch [30/50], Loss: 0.1406, Accuracy: 0.9367, AUC: 0.9793
Epoch [40/50], Loss: 0.1400, Accuracy: 0.9369, AUC: 0.9795
Epoch [50/50], Loss: 0.1395, Accuracy: 0.9371, AUC: 0.9797
Total training time: 1255.87 seconds
PyTorch Base Model Metrics:
{'Experiment': 'Control (Base Model) - PyTorch', 'Learning Rate': 0.001, 'Activation': 'relu', 'num_layers': 2, 'Batch Size': 32, 'Epochs': 50, 'Early Stopping': 'NO', 'Accuracy': 0.9356259234580062, 'Precision': 0.8308373760809956, 'Recall': 0.8200270635994588, 'F1 Score': 0.8253968253968254, 'AUC': np.float64(0.9787392043997466)}


/tmp/ipython-input-2771175341.py:108: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  y_pred_torch_base = torch.sigmoid(torch.tensor(all_test_predictions_raw)).numpy().ravel()


In [ ]:
Base_Model_Metrics_torch

{'Experiment': 'Control (Base Model) - PyTorch',
 'Learning Rate': 0.001,
 'Activation': 'relu',
 'num_layers': 2,
 'Batch Size': 32,
 'Epochs': 50,
 'Early Stopping': 'NO',
 'Accuracy': 0.9356259234580062,
 'Precision': 0.8308373760809956,
 'Recall': 0.8200270635994588,
 'F1 Score': 0.8253968253968254,
 'AUC': np.float64(0.9787392043997466)}

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import numpy as np

# Define the neural network for PyTorch (REGRESSION VERSION)
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.output_layer = nn.Linear(hidden_size2, 1)  # 1 continuous output

    def forward(self, x):
        x = self.relu1(self.layer1(x))
        x = self.relu2(self.layer2(x))
        x = self.output_layer(x)   # no sigmoid!!
        return x

# Select device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Prepare data
X_train_pt = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_pt = torch.tensor(y_train.values, dtype=torch.float32).reshape(-1, 1)

Xtest_pt = torch.tensor(Xtest_scaled, dtype=torch.float32)
ytest_pt = torch.tensor(y_test.values, dtype=torch.float32).reshape(-1, 1)

train_data = TensorDataset(X_train_pt, y_train_pt)
test_data = TensorDataset(Xtest_pt, ytest_pt)

# Model settings
input_size = X_train_scaled.shape[1]
hidden_size1 = 64
hidden_size2 = 32
learning_rate = 0.001
batch_size = 32
epochs = 50

model_torch_reg = SimpleNN(input_size, hidden_size1, hidden_size2).to(device)

# CHANGE 1: Use MSE since this is regression
criterion = nn.MSELoss()
optimizer = optim.Adam(model_torch_reg.parameters(), lr=learning_rate)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Training loop
start_time = time.time()
history_reg = {'loss': []}

model_torch_reg.train()
for epoch in range(epochs):
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_torch_reg(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    history_reg['loss'].append(epoch_loss)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}")

end_time = time.time()
print(f"Total training time: {end_time - start_time:.2f} seconds")

# Evaluation on test data
model_torch_reg.eval()
all_test_labels = []
all_test_predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_torch_reg(inputs)

        all_test_labels.extend(labels.cpu().numpy())
        all_test_predictions.extend(outputs.cpu().numpy())  # raw continuous predictions

# Convert to numpy vectors
y_true = np.array(all_test_labels).ravel()
y_pred = np.array(all_test_predictions).ravel()

# Regression metrics
rmse = mean_squared_error(y_true, y_pred, squared=False)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

Reg_Model_Metrics = {
    "Experiment": "PyTorch Regression Model",
    "Learning Rate": learning_rate,
    "Batch Size": batch_size,
    "Epochs": epochs,
    "RMSE": rmse,
    "MAE": mae,
    "R²": r2
}

print("\nPyTorch Regression Model Metrics:")
print(Reg_Model_Metrics)


Using device: cuda
Epoch [10/50], Loss: 0.0033
Epoch [20/50], Loss: 0.0032
Epoch [30/50], Loss: 0.0032
Epoch [40/50], Loss: 0.0032
Epoch [50/50], Loss: 0.0032
Total training time: 1125.04 seconds


TypeError: got an unexpected keyword argument 'squared'

In [ ]:
rmse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

Reg_Model_Metrics = {
    "Experiment": "PyTorch Regression Model",
    "Learning Rate": learning_rate,
    "Batch Size": batch_size,
    "Epochs": epochs,
    "RMSE": rmse,
    "MAE": mae,
    "R²": r2
}

print("\nPyTorch Regression Model Metrics:")
print(Reg_Model_Metrics)



PyTorch Regression Model Metrics:
{'Experiment': 'PyTorch Regression Model', 'Learning Rate': 0.001, 'Batch Size': 32, 'Epochs': 50, 'RMSE': 0.0032498599030077457, 'MAE': 0.044264569878578186, 'R²': 0.8823034763336182}


In [ ]:
Reg_Model_Metrics

{'Experiment': 'PyTorch Regression Model',
 'Learning Rate': 0.001,
 'Batch Size': 32,
 'Epochs': 50,
 'RMSE': 0.0032498599030077457,
 'MAE': 0.044264569878578186,
 'R²': 0.8823034763336182}